In [1]:
from pathlib import Path
from fast_forward.encoder import TCTColBERTQueryEncoder
from fast_forward.util.pyterrier import FFScore,FFInterpolate
import torch
import pyterrier as pt
import pandas as pd
from pyterrier.measures import RR, nDCG, MAP, AP, R
from fast_forward import OnDiskIndex, Mode, Ranking


In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
index = pt.IndexFactory.of(str(Path.cwd()), memory=True)

In [4]:

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
testset = pt.get_dataset('irds:beir/fiqa/test')
q_encoder = TCTColBERTQueryEncoder("castorini/tct_colbert-msmarco")
ff_index = OnDiskIndex.load(
    Path("ffindex_fiqa_tct.h5"),  query_encoder=q_encoder, mode=Mode.MAXP
)
ff_index = ff_index.to_memory()

100%|██████████| 57638/57638 [00:00<00:00, 1315903.95it/s]


In [5]:

ff_score = FFScore(ff_index)
ff_int = FFInterpolate(alpha=0.1)

In [7]:
topics = testset.get_topics()
sample = topics.sample(100)
print(sample)
#sample.to_csv('queries.csv')

       qid                                              query
439   5592    are hard money loans meant only for real estate
454    879  capital improvement and depreciation in restau...
535   7141  do investors go long option contracts when the...
320   7467             stocks are traded on secondary markets
111   3480      why is okpay not allowed in the united states
..     ...                                                ...
185    988                   where should i invest my savings
64    5150  what credit card information are offline us me...
475  10710  probablity of touching in the money vs expirin...
353   3179  calculation timing and taxes related to profit...
47    3405  non resident aliens question of standard vs it...

[100 rows x 2 columns]


In [8]:
candidates = (bm25 % 100)(sample)
pt.io.write_results(candidates,'trec-run-bm25-fiqa-latency.txt')

In [10]:
%%timeit -r 7 -n 100
candidates = (bm25 % 100)(sample)

1.43 s ± 73.7 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
re_ranked = ff_score(candidates)

In [12]:
%%timeit -r 7 -n 100
re_ranked = ff_score(candidates)

4.09 s ± 138 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
final = ff_int(re_ranked)

In [14]:
%%timeit -r 7 -n 100
final = ff_int(re_ranked)

1.22 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
